# COGS 108 - Final Project 

# Overview

*Fill in your overview here*

# Names

- Karen Rodriguez Pinto
- Enlin Wei
- Yumi Minami 
- Kanami Hannah Tanaka

# Group Members IDs

- A15161856
- A12938737
- A12669124
- A########

# Research Question

*Fill in your research question here*

## Background and Prior Work

*Fill in your background and prior work here* 

References (include links):
- 1)
- 2)

# Hypothesis


*Fill in your hypotheses here*

# Dataset(s)

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name:
- Link to the dataset:
- Number of observations:

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

# Setup

In [1]:
# imports
import pandas as pd
import datetime as dt

In [2]:
# loads Kickstarters data into dataframe
df_kickstarters = pd.read_csv("Kickstarter_projects_Feb19.csv")

In [3]:
# drops column 'id' since it gives no useful information, just an ID associated with the project
# drops column 'currency' since the amounts have already been converted into USD in 'goal_usd' and 'usd_pledged' columns and so is redundant
# drops columns 'start_month' and 'end_month' since already given in 'launched_at' and 'deadline' columns and so is redundant
df_kickstarters.drop(columns=['id', 'currency', 'start_month', 'end_month'], inplace=True)

In [4]:
# method to convert datetime strings in 'df_kickstarters' into datetime objects
def to_datetime(datetime_str):
    return dt.datetime.strptime(datetime_str, '%Y-%m-%d %H:%M:%S')

In [5]:
launched_at_dt_format = []
deadline_dt_format = []
# converts all datetime strings in df to datetime objects and adds them to new lists
# this step may take a while since there are almost 200,000 entries
for i in range(0,len(df_kickstarters)):
    launched_at_dt_format.append(to_datetime(df_kickstarters.iloc[i]['launched_at']))
    deadline_dt_format.append(to_datetime(df_kickstarters.iloc[i]['deadline']))

In [6]:
# adds these lists to the dataframe
df_kickstarters['launched_at_dt_format'] = launched_at_dt_format
df_kickstarters['deadline_dt_format'] = deadline_dt_format

In [7]:
# drops the original 'launched_at' and 'deadline' columns since they are now redundant
df_kickstarters.drop(columns=['launched_at', 'deadline'], inplace=True)

In [8]:
# renames the new columns to the old names
df_kickstarters.rename(columns={'launched_at_dt_format':'launched_at', 'deadline_dt_format':'deadline'}, inplace=True)

In [9]:
df_kickstarters

,name,main_category,sub_category,duration,goal_usd,city,state,country,blurb_length,name_length,status,start_Q,end_Q,usd_pledged,launched_at,deadline
0,Socks of Speed and Socks of Elvenkind,games,Tabletop Games,16.0,2000.000000,Menasha,WI,US,14,7,successful,Q4,Q4,6061.000000,2018-10-30 20:00:02,2018-11-15 17:59:00
1,Power Punch Boot Camp: An All-Ages Graphic Novel,comics,Comic Books,30.0,3870.997710,Shepperton,England,GB,24,8,successful,Q3,Q3,3914.505120,2018-08-06 10:00:43,2018-09-05 10:00:43
2,"Live Printing with SX8: ""Squeegee Pulp Up""",fashion,Apparel,30.0,1100.000000,Manhattan,NY,US,21,7,successful,Q2,Q3,1110.000000,2017-06-09 15:41:03,2017-07-09 15:41:03
3,Lost Dog Street Band's Next Album,music,Country & Folk,45.0,3500.000000,Nashville,TN,US,15,6,successful,Q3,Q4,4807.000000,2014-09-25 18:46:01,2014-11-10 06:00:00
4,"Qto-X, a Tiny Lantern",technology,Gadgets,60.0,30000.000000,Troy,MI,US,15,4,successful,Q4,Q1,40368.000000,2016-11-28 16:35:11,2017-01-27 16:35:11
5,"Aaron McDonnell EP ""Follow""",music,Country & Folk,30.0,7500.000000,Seattle,WA,US,11,4,successful,Q1,Q1,7770.000000,2014-01-22 18:21:37,2014-02-21 18:21:37
6,Lush Life,film & video,Drama,30.0,6000.000000,Los Angeles,CA,US,3,2,successful,Q3,Q3,6500.000000,2018-08-17 18:58:18,2018-09-16 18:58:18
7,The biggest NASA fashion collection in the gal...,design,Graphic Design,30.0,1133.687880,Berlin,Berlin,DE,17,8,successful,Q3,Q3,1168.384000,2018-07-17 21:36:14,2018-08-16 21:36:14
8,The Nothing Box,art,Sculpture,30.0,3000.000000,San Francisco,CA,US,19,3,successful,Q3,Q4,3511.000000,2011-09-17 17:02:56,2011-10-17 17:02:56
9,"""Sittin' 'round waitin' for the world to end"" ...",music,Country & Folk,60.0,1000.000000,Oceanside,CA,US,27,10,successful,Q1,Q2,1770.850000,2013-03-01 17:01:17,2013-04-30 16:01:17


# Data Cleaning

Describe your data cleaning steps here.

In [2]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

# Data Analysis & Results

Include cells that describe the steps in your data analysis.

In [5]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

# Ethics & Privacy

*Fill in your ethics & privacy discussion here*

# Conclusion & Discussion

*Fill in your discussion information here*